# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [15]:
import pandas as pd
import numpy as np
from scipy import stats

In [16]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [17]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

<div class="span5 alert alert-success">
<p>Question 1:
    
As the number of observations is well over the standard lower limit of 30, the Central Limit Theorem applies. As the sample size is large, both a z-test and t-test would likley work, but a z-test is more appropriate to avoid the degrees of freedom aspect of a t-test. An educated guess would be that black-sounding names will receive fewer callbacks so the z-test will be one-tailed.
    
Question 2:

$H_{0}$: There is no difference in callback rates based on race.

$H_{A}$: Black sounding names receive fewer callbacks than white sounding names.</p>
</div>

In [19]:
w = data[data.race=='w']
b = data[data.race=='b']

In [23]:
# Question 3
mean = data.call.mean()
w_mean = w.call.mean()
b_mean = b.call.mean()
std = data.call.std()
w_std = w.call.std()
b_std = b.call.std()
n = len(data.call)
w_n = len(w.call)
b_n = len(b.call)

In [24]:
# Margin of error based on z = +/- 1.96
me = 1.96 * std / np.sqrt(n)
me

0.007641740882124044

In [25]:
# 95% Confidence interval
ci = mean + np.array([-1, 1]) * me
ci

array([0.07285108, 0.08813456])

In [26]:
# Bootstrap approach to confidence interval
def bs_rep(data, func):
    
    """Generates a bootstrap replicate"""
    
    bs_sample = np.random.choice(data, len(data))
    return func(bs_sample)

def draw_bs_reps(data, func, size=1):
    
    """Generates multiple bootstrap replicates."""
    
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = bs_rep(data, func)
    return bs_replicates

bs_reps = draw_bs_reps(data.call, np.mean, 10000)
bs_ci = np.percentile(bs_reps, [2.5, 97.5])
bs_ci

array([0.07310062, 0.08809035])

In [27]:
# z-test
z = (b_mean - w_mean) / np.sqrt(((b_std ** 2) / b_n) + ((w_std ** 2) / w_n))
z

-4.114705323365189

In [28]:
# One-tailed test
p = stats.norm.cdf(z)
p

1.9383717363425055e-05

In [38]:
# Bootstrap method
diff_means = b_mean - w_mean
b_shifted = b.call - b_mean + mean
w_shifted = w.call - w_mean + mean
bs_reps_b = draw_bs_reps(b_shifted, np.mean, 10000)
bs_reps_w = draw_bs_reps(w_shifted, np.mean, 10000)
bs_reps = bs_reps_b - bs_reps_w

p = np.sum(bs_reps <= diff_means) / len(bs_reps)
print('p = %.20f' % p)

p = 0.00000000000000000000


With p-values below 0.05 and outside the 95% confidence interval, we reject the null hypothesis. Having a black-sounding name does negatively affect callback rates.

<div class="span5 alert alert-success">
<p> Question 4:

Discrimination based on race is an every day occurence for many minorities, and exploring how prejudices play out in areas such as employment is vital to take steps towards change. The purpose of this experiment was to determine if having a black-sounding name on a resume was a statistically significant factor in whether or not that resume received a callback from an employer, and the data show that it does. Resumes with black-sounding names received significantly fewer callbacks than resumes with white-sounding names.

Question 5:

This experiment shows that race is a significant factor in regard to callback rates, but it does not show definitively that it is the most important. Various other factors also available in the dataset such as previous work experience, educational experience, volunteer experience, or veteran status could have correlations. Looking at those correlations and seeing how they affect the callback rate would provide a more holistic picture.
</p>
</div>